In [ ]:
import numpy as np
# from biom.table import Table
# from biom import load_table
import pandas as pd
import os

In [ ]:
# Use the TEDreadsTAX reports in TAXprofiles/TEDreadsTAX/reports.
data = "/Volumes/TBHD_share/valencia/pipelines/amos/nibsc/hilo/wgsa/outputs/TAXprofiles/TEDreadsTAX/reports"
output_path = "pipelines/amos/hilo/wgsa"

In [ ]:
tax_dict = {"Genus": "G", "Species": "S", "Family": "F", "Order": "O", "Class": "C", "Phylum": "P", "Kingdom": "K"}

def clean_and_parse_wgsa(data_path, output_dir, rank="Genus"):
    df = pd.read_csv(data_path, sep="\t", header=None, usecols=[0, 3, 5])
    df = df.where(df[3] == tax_dict[rank]).dropna()
    df.sort_values(by=0, ascending=False, inplace=True)

    clean_genus = df[[5, 0]]
    clean_genus.columns = [rank, "RA"]
    clean_genus["RA"] = clean_genus["RA"] / 100
    clean_genus.set_index(rank, inplace=True)

    indices = clean_genus.index
    indices = [i.lstrip() for i in indices]
    clean_genus.index = indices

    # clean_genus.head(10)

    prefix = os.path.basename(data_path).split("_")[0]
    output_file = os.path.join(output_dir, "s" + prefix + "_" + rank.lower() + "_" + "relabund.csv")

    clean_genus.to_csv(output_file, sep=",", header=True, index_label=rank)

# clean_and_parse_wgsa(cami_sim_data, output_path)

In [ ]:
# There may be more than one output file, so we need to combine them.
def combine_files(data_path: str, rank: str):
    if not os.path.exists(data_path):
        raise FileNotFoundError("The data path does not exist.")

    for root, dirs, files in os.walk(data_path):
        print(files)
        if len(files) == 0:
            raise Exception("No files found in output directory.")
            
        for file in files:
            if "REPORT" in file:
                print(os.path.join(root, file))
                clean_and_parse_wgsa(os.path.join(root, file), output_path, rank=rank)

combine_files(data, "Genus")
combine_files(data, "Species")